In [3]:
import os
import traceback
import warnings

os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
warnings.simplefilter("ignore")

import librosa
import soundfile as sf
import os
import time

from multiprocessing import Pool

from tqdm import tqdm


def resample_wav(args):
    input_data_dir, output_data_dir, folder, file, target_sr = args
    audio_folder = os.path.join(output_data_dir, folder)
    out_path_ogg = os.path.join(audio_folder, f"{os.path.splitext(file)[0]}.ogg")
    out_path_wav = os.path.join(audio_folder, f"{os.path.splitext(file)[0]}.wav")
    exists_not_empty_ogg = os.path.exists(out_path_ogg) and os.path.getsize(out_path_ogg) > 20000
    exists_not_empty_wav = os.path.exists(out_path_wav) and os.path.getsize(out_path_wav) > 20000
    if not exists_not_empty_ogg and not exists_not_empty_wav:
        input_path = f'{input_data_dir}/{folder}/{file}'
        try:
            max_duration = 120
            duration_secs = librosa.get_duration(filename=input_path, sr=None)
            duration = max_duration if duration_secs > max_duration else None
            wav, sr = librosa.load(input_path, sr=None, duration=duration)
            wav = librosa.resample(wav, orig_sr=sr, target_sr=target_sr)
            os.makedirs(audio_folder, exist_ok=True)
            try:
                os.remove(out_path_ogg)
            except:
                pass
            sf.write(
                out_path_wav,
                wav,
                target_sr,
                'PCM_32')
        except Exception as e:
            print(f"Cannot process file {input_path}")
            traceback.print_stack()

input_data_dir = 'birdclef-2021/train_short_audio'
output_data_dir = 'train_audio_2021_resampled'

os.makedirs(output_data_dir, exist_ok=True)

folders = os.listdir(input_data_dir)
number_of_folders = len(folders)
number_of_files = {}
for folder in folders:
    number_of_files[folder] = len(os.listdir(f'{input_data_dir}/{folder}'))
    os.makedirs(f'{output_data_dir}/{folder}', exist_ok=True)

os.cpu_count()

target_sr = 32000
search_space = []

for index in range(number_of_folders):
    index_of_folder = index
    folder = folders[index_of_folder]

    for index_of_file in range(number_of_files[folder]):
        file = os.listdir(f'{input_data_dir}/{folder}')[index_of_file]

        search_space.append(
            (
                input_data_dir,
                output_data_dir,
                folder,
                file,
                target_sr
            )
        )

start_time = time.time()
with Pool(processes=12) as pool:
    with tqdm(total=len(search_space), desc="Resampling") as pbar:
        for _ in enumerate(pool.imap_unordered(resample_wav, search_space)):
            pbar.update()

Resampling:  48%|████▊     | 30018/62874 [00:45<17:56, 30.53it/s]  

: 

: 

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import json
import librosa
import soundfile as sf

In [3]:
all_audios = glob.glob('birdclef-2020/train_audio/*/*.ogg')
print(len(all_audios))

21327


In [4]:
input_data_dir = 'birdclef-2020/train_audio'
output_data_dir = 'train_audio_2020_resampled'
os.makedirs(output_data_dir, exist_ok=True)

folders = os.listdir(input_data_dir)
number_of_folders = len(folders)
number_of_files = {}
for folder in folders:
    number_of_files[folder] = len(os.listdir(f'{input_data_dir}/{folder}'))
    os.makedirs(f'{output_data_dir}/{folder}', exist_ok=True)

In [6]:
def process(audio_path):
    wav, sr = librosa.load(audio_path, sr=None, duration=60)
    wav = librosa.resample(wav, orig_sr=sr, target_sr=32000)
    split_arr = audio_path.split('/')
    primary_label = split_arr[2]
    filename = split_arr[3]
#     print(filename)
    out_path_wav = f'{output_data_dir}/{primary_label}/{filename[:-4]}.wav'
    sf.write(
                out_path_wav,
                wav,
                sr,
    'PCM_32')
process(all_audios[0])

In [7]:
_ = Parallel(n_jobs=os.cpu_count())(
    delayed(process)(audio)
    for audio in tqdm(all_audios)
)

  0%|          | 0/21327 [00:00<?, ?it/s]

KeyboardInterrupt: 